In [38]:
from notebookutils import mssparkutils
# mssparkutils.fs.help()

# https://learn.microsoft.com/en-us/fabric/data-engineering/notebook-utilities # Documentation for notebookutils
# https://learn.microsoft.com/en-us/fabric/data-engineering/microsoft-spark-utilities # Documentation for mssparkutils

StatementMeta(, f9ccb846-27f8-4d97-b462-6ead819afb15, 40, Finished, Available, Finished)

# List all file and folders

In [21]:
files = mssparkutils.fs.ls("Files/")
for file in files:
    print(f"Name: {file.name}, Size: {file.size}, Type: {'Folder' if file.isDir else 'File'}")


StatementMeta(, f9ccb846-27f8-4d97-b462-6ead819afb15, 23, Finished, Available, Finished)

Name: 1_calendar_copy.csv, Size: 698286, Type: File
Name: calendar.csv, Size: 698286, Type: File
Name: calendar_copy.csv, Size: 698286, Type: File
Name: test, Size: 0, Type: Folder


# Approach 1: Read files one by one

In [9]:
# Define the Lakehouse path
lakehouse_path = "Files/"  # Update to your folder containing files

# List all files in the Lakehouse directory
files = mssparkutils.fs.ls(lakehouse_path)

# Process files one by one
for file in files:
    if not file.isDir:  # Process only files, skip directories
        file_path = f"{lakehouse_path}/{file.name}"
        print(f"Processing file: {file_path}")

        # Read the file into a DataFrame
        df = spark.read.format("csv").option("header", "true").load(file_path)  # Change format if needed

        # Append data to Delta table
        df.write.format("delta").mode("append").saveAsTable('Test')

print(f"All files appended to Delta table")


StatementMeta(, f9ccb846-27f8-4d97-b462-6ead819afb15, 11, Finished, Available, Finished)

Processing file: Files//calendar.csv
Processing file: Files//calendar_copy.csv
All files appended to Delta table at: Files/delta_table/


# Approach 2: Read all files and then populate them together to delta table

In [17]:
# Define the Lakehouse path
lakehouse_path = "Files/"  # Update to your folder containing files

# List all files in the Lakehouse directory
files = mssparkutils.fs.ls(lakehouse_path)

# Collect file paths into a list
file_paths = [f"{lakehouse_path}/{file.name}" for file in files if not file.isDir]

# Read all files into a single DataFrame
df = spark.read.format("csv").option("header", "true").load(file_paths)  # Change format if needed

# Write the DataFrame to the Delta table
df.write.format("delta").mode("append").saveAsTable('test')

print(f"All files combined and written to Delta table")


StatementMeta(, f9ccb846-27f8-4d97-b462-6ead819afb15, 19, Finished, Available, Finished)

All files combined and written to Delta table


# Validate the counts

In [18]:
df = spark.sql("SELECT * FROM DE_LH_200_LAKE_DevOps_API.dbo.test")
df.count()

StatementMeta(, f9ccb846-27f8-4d97-b462-6ead819afb15, 20, Finished, Available, Finished)

102272

# Approach 1: Read Files One by One Based on Regex

In [35]:
import re

# Define the Lakehouse path
lakehouse_path = "Files/"  # Update to your folder containing files

# Regex pattern to match files starting with a letter
regex_pattern = r"^[a-zA-Z].*"

# List all files in the Lakehouse directory
files = mssparkutils.fs.ls(lakehouse_path)

# Process files one by one
for file in files:
    if not file.isDir and re.match(regex_pattern, file.name):  # Check regex for file name
        file_path = f"{lakehouse_path}/{file.name}"
        print(f"Processing file: {file_path}")

        # Read the file into a DataFrame
        df = spark.read.format("csv").option("header", "true").load(file_path)  # Change format if needed

        # Append data to Delta table
        df.write.format("delta").mode("append").saveAsTable('test_regex')

print(f"All matching files appended to Delta table")


StatementMeta(, f9ccb846-27f8-4d97-b462-6ead819afb15, 37, Finished, Available, Finished)

Processing file: Files//calendar.csv
Processing file: Files//calendar_copy.csv
All matching files appended to Delta table


# Approach 2: Read All Matching Files Together Based on Regex

In [31]:
import re

# Define the Lakehouse path
lakehouse_path = "Files/"  # Update to your folder containing files

# Regex pattern to match files starting with a letter
regex_pattern = r"^[a-zA-Z].*"

# List all files in the Lakehouse directory
files = mssparkutils.fs.ls(lakehouse_path)

# Collect paths of files matching the regex and print matching file names
file_paths = []
for file in files:
    if not file.isDir and re.match(regex_pattern, file.name):  # Check regex for file name
        print(f"Matching file: {file.name}")  # Print file name
        file_paths.append(f"{lakehouse_path}/{file.name}")

# Check if any files matched the criteria
if not file_paths:
    print("No files matched the regex pattern.")
else:
    # Read all matching files into a single DataFrame
    print(f"Processing files: {file_paths}")
    df = spark.read.format("csv").option("header", "true").load(file_paths)  # Change format if needed

    # Write the DataFrame to the Delta table
    df.write.format("delta").mode("overwrite").saveAsTable('test_regex')

    print(f"All matching files combined and written to Delta table")


StatementMeta(, f9ccb846-27f8-4d97-b462-6ead819afb15, 33, Finished, Available, Finished)

Matching file: calendar.csv
Matching file: calendar_copy.csv
Processing files: ['Files//calendar.csv', 'Files//calendar_copy.csv']
All matching files combined and written to Delta table


In [36]:
df = spark.sql("SELECT * FROM DE_LH_200_LAKE_DevOps_API.dbo.test_regex")
df.count()

StatementMeta(, f9ccb846-27f8-4d97-b462-6ead819afb15, 38, Finished, Available, Finished)

153408